In [33]:
# Scripts looks for WordPress websites in Google Search
# Creation date: Sept 2022
# Last modification: 11 Nov 2022

# IDEA: Save non-WordPress sites - maybe I can offer them something too
# IDEA: Drop .gov and .edu domains

from bs4 import BeautifulSoup
import openpyxl
import requests, re
import pandas as pd
import numpy as np
from urllib.parse import urlsplit
from collections import deque
from datetime import datetime, date

search_for = "luxury furniture" #Change only this string
search_for = search_for.replace(" ", "+")
headers = {
    "User-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
}
cities = ["Seattle", "Los Angeles", "Tampa", "Brooklyn", "Phoenix",'Evansville','Springfield ','Independence ','Ann Arbor ','Hartford ','Hillsboro ','Abilene ']
# cities = ['Roanoke','Nampa ','Edinburg ','Clinton ','Federal Way ','New Bedford ','Lees Summit ','Lynn ','Chico ','Quincy ','Davenport ','Wichita Falls ',
# 'Vacaville ','Menifee ','Norwalk ','Spokane Valley ','South Bend ','Woodbridge ','Rialto ','Rio Rancho ','Allen ','Daly City ','Jurupa Valley','Concord ',
# 'Brockton ','San Mateo ','Davie','Tyler ','El Cajon ','Hillsboro ','Renton ','Burbank ','Green Bay ','South Fulton ','Edison ','Inglewood ','Sandy Springs ',
# 'Boulder ','Centennial ','Sparks ','Greeley ','El Monte ','West Covina ','Santa Maria ','Dearborn ','Temecula ','Everett ','Ventura','Murrieta ','Sugar Land ',
# 'Las Cruces ','Miami Gardens ','Lewisville ','Pueblo ','Costa Mesa ','Pompano Beach ','Lakeland ','Lansing ','Peoria ','Broken Arrow ','High Point ','Gresham ',
# 'Downey ','League City ','Springfield ','Waterbury ','Odessa ','Carlsbad ','Elgin ','North Charleston ','Provo ','Beaumont ','Antioch ','Wilmington ','Lowell ',
# 'Manchester ','Westminster ','Richmond ','West Jordan ','Billings ','Clearwater ','Evansville ','West Palm Beach ','Meridian ','Cambridge ','Round Rock ','Richardson ',
# 'Palm Bay ','Fairfield ','Clovis ','College Station ','Hartford ','Lafayette','Rochester ','Independence ','Ann Arbor ','Berkeley ','Arvada ','Abilene ']

def try_except(func, *arg):
    try:
        result = func(*arg)
        print(func.__name__, ":", *arg, "=", result)
        return result
    except Exception as e:
        print(e)
        return "N/a"

def check_if_wordpress(url):
    r = requests.get(url, headers=headers, timeout=5)
    if "wordpress" in r.text:
        return True
    else:
        return False

# def scrap_href(link):
#     links = link.a["href"]
#     links = links.split("/", 3)[:3]
#     links[1] = "//"
#     links[0:3] = [''.join(links[0:3])]
#     return links[0]
       
#Scrapping URLs
is_wordpress = []
links_list = []
for city in cities:
    n = 1
    url = f"https://www.google.com/search?q={search_for}+{city}"
    while n < 10: #number of pages to scrap should be 10
        print(city, n)
        try:
          if n == 1:
              response = requests.get(url, headers=headers, timeout=5)
              soup = BeautifulSoup(response.text, 'lxml')
          else:
              next_page_url = soup.find("a", {"aria-label":f"Page {n}"})["href"]
              response = requests.get(f"https://google.com{next_page_url}", headers=headers)
              soup = BeautifulSoup(response.text, 'lxml')
          for link in soup.find_all("div", {"class":"g"}):
              links = link.a["href"]
              links = links.split("/", 3)[:3]
              links[1] = "//"
              links[0:3] = [''.join(links[0:3])]
              links_list.append(links[0])
              is_wordpress.append(try_except(check_if_wordpress, links[0]))
        except:
          pass
        n += 1

dict = {
    "Website" : links_list,
    "WordPress" : is_wordpress,
    #"Keyword" : search_for,
}

df = pd.DataFrame(dict)
df = df[~df['Website'].isin(['https://allegro.pl', 'https://www.olx.pl', 'https://pl.wikipedia.org', 'https://www.facebook.com', 'https://maps.google.com',
'https://www.yelp.com', 'https://www.indeed.com'])]
df = df.drop_duplicates(subset=['Website'])
df = df[df['WordPress'].isin([True])]
df = df.reset_index(drop=True)
df
now = datetime.now().strftime("%d-%m-%Y_%H-%M")
df.to_excel(rf"C:\Projekty\Coding\wp_security_report\leads\output\g_scrap_{now}.xlsx")


Seattle 1
check_if_wordpress : https://www.viator.com = False
check_if_wordpress : https://www.northtravel.com = False
check_if_wordpress : https://www.tcsworldtravel.com = False
check_if_wordpress : https://www.incognitoglobaltravel.com = False
check_if_wordpress : https://anniewearsit.com = True
check_if_wordpress : https://www.virtuoso.com = False
check_if_wordpress : https://tourtravelandmore.com = True
check_if_wordpress : https://elizabethholmes.com = True
check_if_wordpress : https://www.mrandmrssmith.com = False
check_if_wordpress : https://www.luxurytravelmagazine.com = False
Seattle 2
check_if_wordpress : https://www.forbestravelguide.com = False
check_if_wordpress : https://luxurytravelmodel.com = True
check_if_wordpress : https://www.cooldestinations.com = False
check_if_wordpress : https://www.elegantresorts.co.uk = False
check_if_wordpress : https://www.kiwicollection.com = False
check_if_wordpress : https://www.tripadvisor.com = False
check_if_wordpress : https://www.har

In [34]:
df

,Website,WordPress
0,https://anniewearsit.com,True
1,https://tourtravelandmore.com,True
2,https://elizabethholmes.com,True
3,https://luxurytravelmodel.com,True
4,https://www.aluxurytravelblog.com,True
...,...,...
170,https://www.linearair.com,True
171,https://senterrealtors.com,True
172,https://stabilityhealthcare.com,True
173,https://lgvlimo.com,True
